## The Specific Factors or Ricardo-Viner Model

### Technology and Endowments
Consider a small open economy. There are two sectors Agriculture and Manufacturing.  Production in each sector takes place with a linear homogenous (constant returns to scale) production functions. Agricultural production requires land which is non-mobile or specific to the sector and mobile labor. Manufacturing production requires specific capital and mobile labor.  
$$Q_a = F(\bar T_a, L_a)$$
$$Q_m = G(\bar K_m, L_m)$$
The market for mobile labor is competitive and the market clears at a wage where the sum of labor demands from each sector equals inelastically supplied total labor supply.  
$$L_a + L_m = \bar{L}$$
For the numeric simulations below we make the further assumption that each sector uses a CRS Cobb-Douglas production function:
$$F(T_a, L_a)=\bar T_a^{(1-\alpha)} \cdot  L_a^\alpha$$
$$G(K_m, L_m)=\bar K_m^{(1-\beta)} \cdot  L_m^\beta$$

In [1]:
import numpy as np
from bokeh.plotting import figure, hplot, show, output_file, vplot, output_notebook, gridplot, ColumnDataSource
from bokeh.models import Range1d
from bokeh.models import Slider, CustomJS
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import Latex

output_notebook()
TOOLS = "resize,reset,save,box_select"

BokehJS successfully loaded.

#### Parameters

In [2]:
Tbar = 100       # Fixed specific land in ag. 
Kbar = 100       # Fixed specific capital in manuf
Lbar = 400       # Total number of mobile workers
LbarMax = 400    # Lbar will be on slider, max value.

p    = 1.00      # initial rel price of ag goods, p = Pa/Pm
alpha, beta = 0.5, 0.5  # labor share in ag, manuf

Firms in each sector hires labor so long as the marginal value product of labor is greater than or equal to the market wage:
$$ P_a \cdot MPL_a = w$$
$$ P_m \cdot MPL_m = w$$

Let's also normalize the price of manufacturing to $P_m  =1$  and call $p = \frac{P_a}{P_m}$ the relative price of agricultural goods measured in terms of manufacturing.

### Equilibrium allocations
Let's plot a standard specific factors diagram showing the equilibrium wage and labor allocations.

In [3]:
def Qa(L): 
    return(Tbar**(1-alpha) * L**alpha) 
def Qm(L): 
    return Kbar**(1-beta) * (Lbar - L)**beta
def pMPLa(L):
    return (p * alpha * Qa(L)/L)        # for Cobb-Douglass MPL can be written this way
def MPLm(L):
    return beta * Qm(L)/L
    
La = np.arange(0, LbarMax)
Lm = Lbar - La

**Let's solve for the equilibrium market wage w that clears the market.**

$$p \cdot MPL_a(L_A) = MPL_m(\bar L-L_A) $$

the following function finds La where closest to that intersection $(L_A , w_{eq})$.

In [4]:
from scipy import optimize
def excess_demand(L):
    return pMPLa(L) - MPLm(Lbar - L)

In [5]:
LA = optimize.brentq(excess_demand, 1, Lbar-1)

Simple example with $\bar L$ at $\bar L^{max}$

In [6]:
weq = pMPLa(LA)
titletxt = "Ricardo Viner"
ymax = 1.25
rv = figure(title=titletxt, tools=TOOLS,width=600, height=400)
rv.x_range = Range1d(0, LbarMax)
rv.y_range = Range1d(0, ymax)
rv.line(La, pMPLa(La),line_width=3,legend="Ag LD")
rv.line(La, MPLm(Lbar-La),line_width=3, color = 'red',legend="Mf LD")
rv.line([Lbar,Lbar],[0,ymax], line_width=3, color='black')
rv.line([0,Lbar],[weq,weq], line_width=1, line_dash=[4, 4], color='black')
rv.line([LA, LA],[0, weq], line_width=1, line_dash=[4, 4], color='black')
rv.xaxis.axis_label = 'Labor'
rv.yaxis.axis_label = 'real wage'

print("Real equilibrium wage: ( w/Pm ={0:5.2f}, w/Pa ={1:5.2f} )".format(weq, weq/p))
print("Labor allocations : ( La ={0:5.0f}, Lm ={1:5.0f} )".format(LA, Lbar-LA))
show(rv)

Real equilibrium wage: ( w/Pm = 0.35, w/Pa = 0.35 )
Labor allocations : ( La =  200, Lm =  200 )


## PPF plot
We can plot the associated production possibility frontier.  
It will be useful to draw this as a paremetric plot by defining QM(La) and QA(La) and create a new columndatasource.

In [7]:
QA = lambda La: (Tbar**(1-alpha) * La**alpha)
QM = lambda La: Kbar**(1-beta) * (Lbar - La)**beta

In [8]:
LA, weq = eqn(pMPLa, MPLm)
priceline = (QM(LA)+p*QA(LA))- p*QA(La)
ppfsource = ColumnDataSource(data=dict(
            QA = QA(La), 
            QM = QM(La), 
            priceline = priceline
    )  )

NameError: name 'eqn' is not defined

In [ ]:
titletxt = "PPF"
ppf = figure(title=titletxt, tools=TOOLS,width=500, height=500)
ppf.x_range = Range1d(0, max(Qa)*1.25)
ppf.y_range = Range1d(0, max(Qa)*1.25)
ppf.line('QA','QM', source=ppfsource, line_width=3)
ppf.circle(QA(LA), QM(LA), size=10)
ppf.line('QA','priceline', source=ppfsource, line_width=1, line_dash=[4, 4])
ppf.xaxis.axis_label = 'Qa'
ppf.yaxis.axis_label = 'Qm'

print("At Pa/Pm ={0:5.2f} country produces:".format(p))
print("    Qa ={0:5.0f}, Qm ={1:5.0f} ".format(QA(LA), QM(LA)))
show(ppf)  

## Interactive Plots
We'll use the slider from the ipywidgets library for Ipython working with Bokeh's ability to refresh a graph when a ColumnDataSource is refreshed.
First we define a plot again but this time also give Bokeh a data source that can be refreshed.

In [ ]:
LA, weq = eqn(pMPLa, MPLm)

weqLine = (np.ones(LbarMax)*weq) * (La<Lbar)      # we will use this to plot equilibrium wage

sourceRV = ColumnDataSource(data=dict(
            La = La, 
            pMPLa = pMPLa, 
            MPLm = MPLm,
            weqLine = weqLine
    )  )

rvs = figure(tools=TOOLS, title='Ricardo-Viner', width=600, height=400)
rvs.x_range = Range1d(0, LbarMax)
rvs.y_range = Range1d(0, ymax)
rvs.line('La', 'pMPLa',line_width=3, source=sourceRV,legend="Agric. LD")
rvs.line('La', 'MPLm', line_width=3, source=sourceRV, color='red',legend="Manuf. LD")
rvs.line('La', 'weqLine', line_width=1, line_dash=[4, 4], source=sourceRV, color='black')
rvs.xaxis.axis_label = 'Labor'
rvs.xaxis.axis_label_text_font_size = "12pt"
rvs.yaxis.axis_label = r'Real wage'
rvs.yaxis.axis_label_text_font_size = "12pt"
rvs.legend.orientation = "top_right"

This next function takes slider values for $\bar L$ and $p=\frac{P_a}{P_m}$, updates the curves to be plotted and pushes those values to later Bokeh plots' data 'source' for a refresh. 

In [ ]:
def updateRV(Lbar=LbarMax, p=1, Tbar=100,Kbar=100,):
    Qa = (Tbar**(1-alpha) * La**alpha) * (La<Lbar)
    Qm = Kbar**(1-beta) * (Lbar - La)**beta
    pMPLa = (p * alpha * Qa/La) *(La<Lbar)        
    MPLm = beta * Qm/(Lbar-La)
    
    LA, weq = eqn(pMPLa, MPLm)
    weqLine = (np.ones(LbarMax)*weq) * (La<Lbar) 
    
    QA = (Tbar**(1-alpha) * LA**alpha)
    QM = Kbar**(1-beta) * (Lbar - LA)**beta
    sourceRV.data['MPLm'] = MPLm
    sourceRV.data['pMPLa'] = pMPLa
    sourceRV.data['weqLine'] = weqLine  
    sourceRV.push_notebook()
    print("Equilibrium wages: ( w/Pm ={0:5.2f}, w/Pa ={1:5.2f} )".format(weq, weq/p))
    print("Labor and output : ( La ={0:5.0f}, Lm ={1:5.0f} ), \
      (Qa = {2:5.0f}, Qm = {3:5.0f}) ".format(LA, Lbar-LA, QA, QM))

Now let's display the plot and follow that by a slider.

In [ ]:
rvs.plot_width=500
show(rvs)

In [ ]:
interact(updateRV,Lbar=(100, LbarMax, 50), p =(0.25,2,0.25), Tbar=(50, 200, 50), Kbar=(50, 200, 50))

## Two synchronized plots
Let's try to show the labor allocation and PPF plots side by side and responsive to the same slider.  For this we are going to setup two data sources, one for each plot.  

In [ ]:
def updateBoth(Lbar=LbarMax, p=1, Tbar=100,Kbar=100,):
    Qa = (Tbar**(1-alpha) * La**alpha) * (La<Lbar)
    Qm = Kbar**(1-beta) * (Lbar - La)**beta
    pMPLa = (p * alpha * Qa/La) *(La<Lbar)        
    MPLm = beta * Qm/(Lbar-La)
    
    LA, weq = eqn(pMPLa, MPLm)
    weqLine = (np.ones(LbarMax)*weq) * (La<Lbar) 
    
    sourceRV.data['MPLm'] = MPLm
    sourceRV.data['pMPLa'] = pMPLa
    sourceRV.data['weqLine'] = weqLine  
    sourceRV.push_notebook()
    
    QA = lambda La: (Tbar**(1-alpha) * La**alpha)
    QM = lambda La: Kbar**(1-beta) * (Lbar - La)**beta
    priceline = (QM(LA)+p*QA(LA)) - p*QA(La)
    
    ppfsource.data['QA'] = QA(La)
    ppfsource.data['QM'] = QM(La)
    ppfsource.data['priceline'] = priceline  
    ppfsource.push_notebook()
   

As of 10/1/15 THe plots that follow are not quite working.

Major issue: only the RV plot is updating... PPF plot is not changed, not sure why.
Minor: need to rescale to fit the two plots side by side.

In [ ]:
ppf2=ppf
rvs2=rvs

ppf2.plot_height=350
ppf2.plot_width=350
rvs2.plot_height=350
rvs2.plot_width=350
show(vplot(hplot(ppf, rvs,)))

In [ ]:
interact(updateBoth,Lbar=(100, LbarMax, 50), p =(0.25,2,0.25), Tbar=(50, 200, 50), Kbar=(50, 200, 50))